In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import tensorflow as tf

#import keras
from tensorflow import keras

import keras.optimizers as tko
import keras.activations as tka
import keras.losses as tkl
from keras.layers import Input, Dense
from keras.models import Model

from sklearn.model_selection import train_test_split
from sklearn import metrics

import commonmodules as cm

#import fittingviaNN_3D_CLI as f3d
from sklearn.preprocessing import MinMaxScaler
import pickle

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import fittingviaNN_process_one as f3d

filename = "N2H2_VV_process.xlsx"
testtovisualize = False
wselected = 0
columselected = 1

df = pd.read_excel(filename)

x = df[['v', 'w', 'T']].values
y = df[['RC']].values
usedlog10 = True
if usedlog10:
    y = np.log10(y)
scalery = MinMaxScaler()
scalery.fit(y)
y_s = scalery.transform(y)

scalerx = MinMaxScaler()
scalerx.fit(x)
x_s = scalerx.transform(x)

vmap_toreal = {}

for i, vn in enumerate(x_s[:,0]):
    vmap_toreal[vn] = x[i,0]

print("V map: ")
for a in vmap_toreal:
    print("%4.2f --> %3d"%(a, vmap_toreal[a]))

vset = set(x_s[:,0])
wset = set(x_s[:,1])
tset = set(x_s[:,2])
vlist = list(vset)

In [ ]:
# chek data
#for w in wset:
#    train_x, test_x, train_y, test_y = cm.test_train_split (1, [w], x_s, y_s)
#
#    cm.plotfull3dcurve (1, test_x, test_y)

In [ ]:
# so all the models should start from the same point
#np.random.seed(812)
#keras.utils.set_random_seed(812)
#tf.config.experimental.enable_op_determinism()
import time

modelshape_s = [
        [256, 256, 256], 
        [256, 256, 256, 256, 256, 256]]
batch_size_s = [50, 256]
epochs_s = [1000]
lossfuns = ['mse']
optimizers = ['adam']
activations = ['relu']

# run a grid search
totalnum = len(modelshape_s)*\
        len(batch_size_s)*len(epochs_s)*\
            len(lossfuns)*len(optimizers)*len(activations)
print("Total number of models to run: ", totalnum)
modelnum = 0
for modelshape in modelshape_s:
    for batch_size in batch_size_s:
        for epochs  in epochs_s:
            for lossfun in lossfuns:
                for optimizer in optimizers:
                    for activation in activations:
                        modelnum += 1

                        r2test_v_split = 0.0
                        msetest_v_split = 0.0
                        r2train_v_split = 0.0 
                        msetrain_v_split = 0.0
                        
                        r2test_vsets_split = 0.0
                        msetest_vsets_split = 0.0
                        r2train_vsets_split = 0.0 
                        msetrain_vsets_split = 0.0

                        timestart = time.time()

                        r2train_v_split, msetrain_v_split, r2test_v_split, msetest_v_split = \
                            f3d.build_v_split (x_s, y_s, scalery, scalerx, False, vset, modelshape, \
                                        batch_size, epochs, \
                                        lossfun, optimizer, activation, \
                                        vmap_toreal, modelfname="vsplitmodel_"+str(modelnum)+".csv")
                        
                        r2train_vsets_split, msetrain_vsets_split, \
                            r2test_vsets_split, msetest_vsets_split = \
                            f3d.build_vsets_split (x_s, y_s, scalery, scalerx, False, vlist, modelshape, \
                                            batch_size, epochs, \
                                            lossfun, optimizer, activation, \
                                            vmap_toreal, modelfname="vsetsplitmodel_"+str(modelnum)+".csv")
                        
                        print("v split , Model metrics %3d , %10.5f , %10.5f , %10.5f , %10.5f"%( \
                            modelnum, r2test_v_split, msetest_v_split, \
                            r2train_v_split, msetrain_v_split), flush=True)
                        print("vsets split , Model metrics %3d , %10.5f , %10.5f , %10.5f , %10.5f"%( \
                            modelnum, r2test_vsets_split, msetest_vsets_split, \
                            r2train_vsets_split, msetrain_vsets_split), flush=True)
                        print("Model shapes  %3d , %s , %5d , %5d , %s , %s , %s "%( \
                             modelnum, str(modelshape), batch_size, epochs, \
                                lossfun, optimizer, activation), flush=True)
                        
                        timeend = time.time()
                        print("Total seconds taken: ", timeend-timestart, flush=True)